In [1]:
import os
import random

import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, Subset

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import VGG19_Weights

# Project utilities
import utils
import preprocessing

# Set seed
SEED = 42
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
np.random.seed(SEED)

##############
# torch.manual_seed(20220215)
# torch.cuda.manual_seed(20220215)
# torch.backends.cudnn.deterministic = True
# torch.use_deterministic_algorithms = True
##########

ROOT = './data'
DATASET_DIR = './data/Post_Impressionism'
CSV_FILE = './data/classes.csv'

if not (os.path.exists(DATASET_DIR) and os.path.exists(CSV_FILE)):
    raise FileExistsError("File doesn't exist, we expect a data folder with the labels and the images folder as in the lab")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(), # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229,0.224, 0.225])]) # Normalize the pixel values based on ImageNet statistics, to a range that VGG expects

dataset = preprocessing.ImageFolderForBinaryClassification(root=ROOT, target='is_van_gogh', transform=transform)
pics_in_csv = [i for i in range(len(dataset)) if dataset.samples[i][1] >= 0]
dataset = Subset(dataset, pics_in_csv)

In [22]:


classes = pd.read_csv(CSV_FILE)
train_classes = classes[classes['subset'] == 'train']
test_classes = classes[classes['subset'] == 'test']

train_indices, valid_indices = train_test_split(train_classes.index.to_list(), test_size=0.2, random_state=SEED)
train_dataset = Subset(dataset, train_indices)
valid_dataset = Subset(dataset, valid_indices)

In [20]:
hist = dict()
hist['debug'] = []
for path, label in dataset.samples:
    if label not in hist:
        hist[label] = 0
    hist[label] += 1
    if label == -1:
        hist['debug'].append(path)

hist

{'debug': [], 0: 3547, 1: 1004}

In [8]:
from torchvision.transforms.functional import to_pil_image
to_pil_image(image_tensor.squeeze()).show()

# Fine tuning VGG19

In [11]:
# Load pre-trained VGG19 model
# vgg19 = models.vgg19(pretrained=True)
vgg19 = models.vgg19(weights=models.VGG19_Weights.DEFAULT)


In [16]:
models.VGG19_Weights.DEFAULT

VGG19_Weights.IMAGENET1K_V1

# Fine tuning AlexNet

In [ ]:
# Load the AlexNet model 
alexnet = models.alexnet(pretrained=True)


analysing results

# Style transfer function